<a href="https://colab.research.google.com/github/swilsonmfc/pandas/blob/main/4_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution 4

# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

# Data

In [ ]:
red_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
red_df['wine'] = 'red'
red_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,red
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,red
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,red
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,red


In [ ]:
white_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
white_df['wine'] = 'white'
white_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


In [ ]:
wine_df = pd.concat([red_df, white_df])
wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


# Bin Alchohol Content
* Add a column with a label indicating the alcohol content
  * 25th Percentile -> Low
  * 25 - 75th Percentile -> Normal
  * 75th Percentile 

In [ ]:
# Describe it & use the values to set thresholds
print(wine_df.alcohol.describe())
wine_df['alcohol_label'] = np.where(wine_df.alcohol <= 9.5 , 'Low',
                           np.where(wine_df.alcohol <= 11.3, 'Normal', 'High'))

count    6497.000000
mean       10.491801
std         1.192712
min         8.000000
25%         9.500000
50%        10.300000
75%        11.300000
max        14.900000
Name: alcohol, dtype: float64


In [ ]:
# Extract programmatically - better
_25 = wine_df.alcohol.quantile(.25)
_75 = wine_df.alcohol.quantile(.75)

wine_df['alcohol_label'] = np.where(wine_df.alcohol <= _25, 'Low',
                           np.where(wine_df.alcohol <= _75, 'Normal', 'High'))

In [ ]:
# An alternative to annoying nested np.where & slow apply
_25 = wine_df.alcohol.quantile(.25)
_75 = wine_df.alcohol.quantile(.75)

alc_low  = (wine_df.alcohol <= _25)
alc_norm = (wine_df.alcohol >  _25) & (wine_df.alcohol <= _75)
alc_high = (wine_df.alcohol >  _75)

wine_df['alcohol_label'] = np.select([alc_low, alc_norm, alc_high], ['Low', 'Normal', 'High'])
wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine,alcohol_label
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red,Low
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red,Normal
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red,Normal
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red,Normal
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white,Normal
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white,Normal
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white,Low
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white,High


# Grouping Sugars
* Generally speaking, what's the relationship between sugar and the alcohol label

In [ ]:
# Alcohol and Sugar appear to have a inverse relationship
wine_df.groupby('alcohol_label')['residual sugar'].mean()

alcohol_label
High      3.742721
Low       8.252831
Normal    4.596363
Name: residual sugar, dtype: float64

# Crosstabbing Wine & Alcohol
* Use a crosstab table to show the combinations of wine and alcohol 

In [ ]:
# Show counts 
pd.crosstab(wine_df.wine, wine_df.alcohol_label, margins=True)

alcohol_label,High,Low,Normal,All
wine,,,,
red,312,436,851,1599
white,1261,1436,2201,4898
All,1573,1872,3052,6497


# Aggregate Sulphates
* What is the 10th and 90th percentiles for sulphates by wine type

In [ ]:
# Avoid the temptation
for wine in ['red', 'white']:
  for percentile in [.1, .9]: 
    value = wine_df[wine_df.wine == wine]['sulphates'].quantile(percentile)
    print(f'{wine} - {percentile} == {value}')

red - 0.1 == 0.5
red - 0.9 == 0.85
white - 0.1 == 0.36
white - 0.9 == 0.64


In [ ]:
# Quantile isn't available directly
# Use a function you pass in or a lambda
# Pass in a tuple (column level 1 name, lambda) to format
wine_df.groupby('wine').agg({'sulphates': [(0.1, lambda x: x.quantile(0.1)),
                                           (0.9, lambda x: x.quantile(0.9))]})

sulphates      
            0.1   0.9
wine                 
red        0.50  0.85
white      0.36  0.64

# Pivot on Quality
* Given wine (red / white) and alcohol, what's the mean and median quality

In [ ]:
# Create a pivot table
wine_df.pivot_table(values='quality', index=['wine', 'alcohol_label'], aggfunc=[np.mean, np.median])

mean  median
                      quality quality
wine  alcohol_label                  
red   High           6.282051       6
      Low            5.256881       5
      Normal         5.593420       6
white High           6.421094       6
      Low            5.493036       5
      Normal         5.817810       6